In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=0.)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.98 sec - Loss 0.105701 - ACC 83.54% - ACC Mean 83.54% - AUC 90.79% - AUC Mean 90.79% - Deter 000
Ite 00050 - 0.13 sec - Loss 0.098855 - ACC 83.21% - ACC Mean 83.93% - AUC 90.42% - AUC Mean 90.70% - Deter 042
Ite 00100 - 0.09 sec - Loss 0.081510 - ACC 84.40% - ACC Mean 84.24% - AUC 90.86% - AUC Mean 90.56% - Deter 092
Ite 00150 - 0.10 sec - Loss 0.051500 - ACC 84.82% - ACC Mean 83.64% - AUC 91.07% - AUC Mean 90.68% - Deter 142
Ite 00200 - 0.10 sec - Loss 0.031194 - ACC 84.25% - ACC Mean 84.47% - AUC 92.13% - AUC Mean 91.57% - Deter 000
Ite 00250 - 0.10 sec - Loss 0.024782 - ACC 83.39% - ACC Mean 84.18% - AUC 91.84% - AUC Mean 91.68% - Deter 050
Ite 00300 - 0.11 sec - Loss 0.022782 - ACC 83.41% - ACC Mean 83.96% - AUC 90.89% - AUC Mean 91.47% - Deter 100
Ite 00350 - 0.10 sec - Loss 0.021857 - ACC 87.47% - ACC Mean 84.31% - AUC 92.58% - AUC Mean 91.57% - Deter 000
Ite 00400 - 0.11 sec - Loss 0.021241 - ACC 85.00% - ACC Mean 84.89% - AUC 92.42% - AUC Mean 92.09% - Deter 006
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 87.98% +- 0.0% - auc: 92.43% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 86.05% +- 4.14% - auc: 91.0% +- 3.65%
SFIL RUN 2/10 - acc: 85.57% +- 2.84% - auc: 90.1% +- 1.4%
SFIL RUN 3/10 - acc: 84.32% +- 3.13% - auc: 91.92% +- 3.41%
SFIL RUN 4/10 - acc: 84.43% +- 5.24% - auc: 91.41% +- 2.89%
SFIL RUN 5/10 - acc: 84.03% +- 1.75% - auc: 88.87% +- 1.99%
SFIL RUN 6/10 - acc: 87.2% +- 3.88% - auc: 92.69% +- 3.48%
SFIL RUN 7/10 - acc: 85.12% +- 5.35% - auc: 90.62% +- 2.58%
SFIL RUN 8/10 - acc: 85.19% +- 3.11% - auc: 91.44% +- 2.91%
SFIL RUN 9/10 - acc: 84.25% +- 2.95% - auc: 90.24% +- 3.89%
SFIL RUN 10/10 - acc: 87.01% +- 2.77% - auc: 90.88% +- 2.53%
SFIL GLOBAL - acc: 85.32% +- 1.08% - auc: 90.92% +- 1.0%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 84.02% +- 2.17% - auc: 88.89% +- 3.71%
PFIL RUN 2/10 - acc: 82.43% +- 5.35% - auc: 89.04% +- 4.29%
PFIL RUN 3/10 - acc: 84.8% +- 2.98% - auc: 90.76% +- 2.19%
PFIL RUN 4/10 - acc: 82.2% +- 7.36% - auc: 88.66% +- 8.66%
PFIL RUN 5/10 - acc: 87.06% +- 1.79% - auc: 91.36% +- 1.15%
PFIL RUN 6/10 - acc: 86.44% +- 5.65% - auc: 91.35% +- 4.71%
PFIL RUN 7/10 - acc: 86.52% +- 5.81% - auc: 91.7% +- 3.44%
PFIL RUN 8/10 - acc: 84.36% +- 1.82% - auc: 91.11% +- 3.73%
PFIL RUN 9/10 - acc: 80.83% +- 5.46% - auc: 89.37% +- 1.85%
PFIL RUN 10/10 - acc: 84.19% +- 2.81% - auc: 90.66% +- 2.62%
PFIL GLOBAL - acc: 84.28% +- 1.93% - auc: 90.29% +- 1.11%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 85.27% +- 2.4% - auc: 90.13% +- 3.92%
SPOL RUN 2/10 - acc: 84.77% +- 6.47% - auc: 90.57% +- 2.86%
SPOL RUN 3/10 - acc: 87.33% +- 4.12% - auc: 92.67% +- 2.74%
SPOL RUN 4/10 - acc: 80.3% +- 1.84% - auc: 89.12% +- 3.0%
SPOL RUN 5/10 - acc: 84.44% +- 2.97% - auc: 90.7% +- 2.03%
SPOL RUN 6/10 - acc: 87.57% +- 6.1% - auc: 93.45% +- 4.31%
SPOL RUN 7/10 - acc: 84.72% +- 1.49% - auc: 91.97% +- 1.27%
SPOL RUN 8/10 - acc: 84.33% +- 4.85% - auc: 90.6% +- 3.63%
SPOL RUN 9/10 - acc: 86.51% +- 3.27% - auc: 92.23% +- 2.13%
SPOL RUN 10/10 - acc: 85.02% +- 4.01% - auc: 92.25% +- 1.54%
SPOL GLOBAL - acc: 85.03% +- 1.93% - auc: 91.37% +- 1.27%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 83.18% +- 1.72% - auc: 91.64% +- 1.35%
PPOL RUN 2/10 - acc: 83.6% +- 2.8% - auc: 89.71% +- 3.46%
PPOL RUN 3/10 - acc: 84.71% +- 2.75% - auc: 91.23% +- 0.93%
PPOL RUN 4/10 - acc: 84.93% +- 5.37% - auc: 90.26% +- 6.45%
PPOL RUN 5/10 - acc: 85.48% +- 3.23% - auc: 91.38% +- 1.91%
PPOL RUN 6/10 - acc: 83.24% +- 2.76% - auc: 89.71% +- 2.86%
PPOL RUN 7/10 - acc: 84.93% +- 5.24% - auc: 90.99% +- 4.97%
PPOL RUN 8/10 - acc: 83.92% +- 6.29% - auc: 92.0% +- 4.76%
PPOL RUN 9/10 - acc: 84.25% +- 5.53% - auc: 90.06% +- 2.38%
PPOL RUN 10/10 - acc: 84.88% +- 3.16% - auc: 91.34% +- 5.04%
PPOL GLOBAL - acc: 84.31% +- 0.76% - auc: 90.83% +- 0.79%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly worse than MICE
Metric AUC - OUR METHOD is even with MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
